In [1]:
!pip install -q langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 28.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [2]:
!pip install -q langchain

In [3]:
!pip install -q langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [8]:
import os
from google.colab import userdata

In [7]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "chatbot_with_langchain"
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [10]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [11]:
model.invoke("hi").content

'Hi there! How can I help you today?'

In [12]:
from langchain_core.output_parsers import StrOutputParser

In [13]:
parser = StrOutputParser()

In [14]:
parser.invoke(model.invoke("hi"))

'Hi there! How can I help you today?'

In [15]:
from langchain_core.messages import HumanMessage

In [16]:
import warnings
warnings.filterwarnings("ignore")

In [17]:
while True:
  message = input("Write your query:")
  if message=="bye":
    print("Good Bye have a great day!")
    break
  else:
    print(parser.invoke(model.invoke([HumanMessage(content=message)])))

Write your query:hello
Hello! How can I help you today?
Write your query:bye
Good Bye have a great day!


In [18]:
from langchain_core.chat_history import BaseChatMessageHistory

In [19]:
from langchain_core.chat_history import InMemoryChatMessageHistory

In [20]:
from langchain_core.runnables.history import RunnableWithMessageHistory

In [21]:
from langchain_core.messages import AIMessage

In [22]:
result = model.invoke(
    [
        HumanMessage(content="Hi! I'm dnb"),
        AIMessage(content="Hello dnb! How can I assist you today?"),
        HumanMessage(content="What's my name?")
    ]
)

In [23]:
parser.invoke(result)

'Your name is dnb!'

In [24]:
store={}

In [25]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [26]:
config = {"configurable": {"session_id": "firstchat"}}

In [27]:
model_with_memory=RunnableWithMessageHistory(model,get_session_history)

In [28]:
model_with_memory.invoke([HumanMessage(content="Hi! I'm dnb")],config=config).content

"Hello dnb! Nice to meet you.\n\nI'm an AI, a large language model. How can I assist you today?"

In [29]:
model_with_memory.invoke([HumanMessage(content="what is my name?")],config=config).content

'Based on what you told me, your name is **dnb**.'

In [30]:
config = {"configurable": {"session_id": "secondtchat"}}

In [31]:
config

{'configurable': {'session_id': 'secondtchat'}}

In [32]:
model_with_memory.invoke([HumanMessage(content="what is my name?")],config=config).content

"As an AI, I don't have access to your personal information, including your name. I only know what you tell me during our conversation.\n\nIf you'd like to tell me your name, I'd be happy to remember it for the duration of our chat!"

In [33]:
config = {"configurable": {"session_id": "firstchat"}}

In [34]:
config

{'configurable': {'session_id': 'firstchat'}}

In [35]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm dnb", additional_kwargs={}, response_metadata={}), AIMessage(content="Hello dnb! Nice to meet you.\n\nI'm an AI, a large language model. How can I assist you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--b94e01cd-1ccb-4370-90c2-a0ec10a74942-0', usage_metadata={'input_tokens': 8, 'output_tokens': 697, 'total_tokens': 705, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 669}}), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Based on what you told me, your name is **dnb**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--7

In [36]:
model_with_memory.invoke([HumanMessage(content="what is my name?")],config=config).content

'Your name is **dnb**.'

In [37]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [38]:
prompt=ChatPromptTemplate.from_messages(
    [
      ("system","You are a helpful assistant. Answer all questions to the best of your ability.",),
      MessagesPlaceholder(variable_name="messages"),
    ]
)

In [39]:
chain = prompt | model

In [40]:
chain.invoke({"messages": ["hi! I'm jack"]})

AIMessage(content="Hi Jack! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--6ac0a33a-7572-49b7-ace9-35ca66082f0d-0', usage_metadata={'input_tokens': 23, 'output_tokens': 52, 'total_tokens': 75, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 34}})

In [41]:
model_with_memory=RunnableWithMessageHistory(chain,get_session_history)

In [42]:
config = {"configurable": {"session_id": "thirdchat"}}

In [43]:
response=model_with_memory.invoke(
    [HumanMessage(content="Hi! I'm Jim"),
     ],config=config
)

In [44]:
response.content

"Hi Jim! It's nice to meet you. How can I help you today?"

In [45]:
response=model_with_memory.invoke(
    [HumanMessage(content="hi what is my name"),
     ],config=config
)
print(response.content)

Your name is Jim.


In [46]:
response=model_with_memory.invoke(
    [HumanMessage(content="what is 2+2?"),
     ],config=config
)
print(response.content)

2 + 2 = 4.


In [47]:
response=model_with_memory.invoke(
    [HumanMessage(content="what should i add in previous addition so that i will become 10?"),
     ],config=config
)
print(response.content)

You should add **6** to 4 (the result of 2+2) to get 10.


In [48]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm dnb", additional_kwargs={}, response_metadata={}), AIMessage(content="Hello dnb! Nice to meet you.\n\nI'm an AI, a large language model. How can I assist you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--b94e01cd-1ccb-4370-90c2-a0ec10a74942-0', usage_metadata={'input_tokens': 8, 'output_tokens': 697, 'total_tokens': 705, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 669}}), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Based on what you told me, your name is **dnb**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--7

## trimming the conversation

In [49]:
from langchain_core.messages import SystemMessage, trim_messages

In [56]:
trimmer = trim_messages(
    max_tokens=50,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [57]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [58]:
model.get_num_tokens_from_messages(messages)

51

In [59]:
trimmer.invoke(messages)

[HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [60]:
trimmed_message = trimmer.invoke(messages)

In [61]:
model.get_num_tokens_from_messages(trimmed_message)

39

In [62]:
prompt

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.mes

In [63]:
prompt=ChatPromptTemplate.from_messages(
    [
      ("system","You are a helpful assistant. Answer all questions to the best of your ability.",),
      MessagesPlaceholder(variable_name="messages"),
    ]
)

In [64]:
trimmed_message+[HumanMessage(content="what's my name?")]

[HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={})]

In [65]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I don't know your name! You haven't told it to me yet.\n\nWould you like to tell me what it is?"

In [66]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2".'